In [34]:
import pandas as pd
from tqdm.notebook import tqdm
from surprise import KNNWithZScore, SVDpp
from surprise import Dataset, accuracy, Reader
from surprise.model_selection import train_test_split, GridSearchCV


In [17]:
movies = pd.read_csv('C:/Users/User/Desktop/Data Scientist/AML/Recsys/data/ml-1m/movies.dat',
                     sep='::', names=['movieId', 'title', 'genres'], engine='python')
ratings = pd.read_csv('C:/Users/User/Desktop/Data Scientist/AML/Recsys/data/ml-1m/ratings.dat', 
                     sep='::', names=['userId', 'movieId', 'rating', 'timestamp'], engine='python')
users = pd.read_csv('C:/Users/User/Desktop/Data Scientist/AML/Recsys/data/ml-1m/users.dat', 
                   sep='::', names=['userId', 'gender', 'age', 'occupation', 'zipcode'], engine='python')

In [12]:
movies.head()

,movieId,title,gener
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [ ]:
movies.shape

In [40]:
movies.to_csv('movies.csv')

In [19]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [ ]:
ratings.shape

In [41]:
ratings.to_csv('ratings.csv')

In [42]:
df = pd.read_csv('ratings.csv')
df

,Unnamed: 0,userId,movieId,rating,timestamp
0,0,1,1193,5,978300760
1,1,1,661,3,978302109
2,2,1,914,3,978301968
3,3,1,3408,4,978300275
4,4,1,2355,5,978824291
...,...,...,...,...,...
1000204,1000204,6040,1091,1,956716541
1000205,1000205,6040,1094,5,956704887
1000206,1000206,6040,562,5,956704746
1000207,1000207,6040,1096,4,956715648


In [21]:
len(movies.movieId.unique()), len(ratings.movieId.unique())

(3883, 3706)

In [23]:
movies_with_ratings = pd.merge(ratings, movies, left_on='movieId', right_on='movieId')

In [25]:
movies_with_ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
1000204,5949,2198,5,958846401,Modulations (1998),Documentary
1000205,5675,2703,3,976029116,Broken Vessels (1998),Drama
1000206,5780,2845,1,958153068,White Boys (1999),Drama
1000207,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western


In [ ]:
movies_with_ratings.shape

In [35]:
dataset = pd.DataFrame({'uid': movies_with_ratings.userId, 
                       'iid': movies_with_ratings.title, 
                       'rating': movies_with_ratings.rating})

scale_min = ratings.rating.min()
scale_max = ratings.rating.max()

reader = Reader(rating_scale=(scale_min, scale_max))
data = Dataset.load_from_df(dataset, reader)

In [36]:
trainset, testset = train_test_split(data, test_size=.2)

In [38]:
algo = SVDpp()
algo.fit(trainset)
pred_test = algo.test(testset)
accuracy.rmse(pred_test, verbose=True)

KeyboardInterrupt: 

In [ ]:
data = Dataset.load_builtin('ml-1m')